In [8]:
import os
import sys
import numpy as np
current_path = os.getcwd()

sys.path.append(os.path.join(current_path,'../motion_planning'))


from motion_planning import motion_planner_agv
from agv_kinematics import agv_arm,Jaka_FK

from jaka_IK import get_DH_matrix_JAKA_ZU7,forward_kinematic_DH_solution,inverse_kinematic_DH_solution,JAKA_inverse_kinematic_DH_solution,get_DH_matrix_JAKA_ZU7_ur


In [9]:
from math import pi
class jaka_zu7_IK():
    def __init__(self):
        self.DH_matrix_JAKA_ZU7 = get_DH_matrix_JAKA_ZU7()
        self.DH_matrix_JAKA_ZU7_ur = get_DH_matrix_JAKA_ZU7_ur()
        self.lower_bound = np.array([-3.14,-3.14,-3.14,-3.14,-6.28,-6.28])
        self.upper_bound = -np.array([-3.14,-3.14,-3.14,-3.14,-6.28,-6.28])
    
    def FK_JAKA(self, q):
        return forward_kinematic_DH_solution(self.DH_matrix_JAKA_ZU7, q)
    
    def IK(self, T_0_6):
        q_list = []

        res = JAKA_inverse_kinematic_DH_solution(self.DH_matrix_JAKA_ZU7, self.DH_matrix_JAKA_ZU7_ur, T_0_6)
        for next_q in res.T:
            trans_ = self.FK_JAKA(next_q)
            error = np.linalg.norm(trans_-T_0_6)
            if error < 1e-6:
                q_list.append(next_q)
                #do not exceed the bound
                for i in range(6):
                    if next_q[i] > self.upper_bound[i]:
                        next_q[i] = self.upper_bound[i]
                    elif next_q[i] < self.lower_bound[i]:
                        next_q[i] = self.lower_bound[i]
        return np.array(q_list)
        


ed = np.asarray([3.315783, 2.240158, -2.179694, 1.983942, 1.634298, 1.492484])
# ed = np.asarray([3.315783, 2.240158, 1, 1.983942, 1.634298, 0.01])
# ed = np.asarray([0,0,0,0,0,0.0])


jaka_ik_cal = jaka_zu7_IK()
T_0_6J = jaka_ik_cal.FK_JAKA(ed)
qs = jaka_ik_cal.IK(T_0_6J)
print(qs)


[[ 3.14        2.240158   -2.179694    1.983942    1.634298    1.492484  ]
 [ 3.14        0.3839884   2.179694   -0.5192764   1.634298    1.492484  ]
 [ 3.14        1.62251331 -1.47477888 -1.24492109 -1.634298   -1.64910865]
 [ 3.14        0.30212104  1.47477888 -2.87408657 -1.634298   -1.64910865]
 [ 1.30841416  2.90142051 -1.52907292 -0.07591349  1.96671537 -0.5855818 ]
 [ 1.30841416  1.53533314  1.52907292 -1.76797198  1.96671537 -0.5855818 ]
 [ 1.30841416  2.69658362 -2.11507942 -2.42666276 -1.96671537  2.55601085]
 [ 1.30841416  0.88139428  2.11507942  1.44155305 -1.96671537  2.55601085]]


In [11]:
import copy
#agv arm IK
now_agv_arm = agv_arm()
q = np.array([0,0,0, 1, 1, 1, 0, 0, 1])
q = np.array([0,0,0, 0, 0, 0, 0, 0, 0])
T_fk = now_agv_arm.FK(q)
# now_agv_arm.vis_collision()

#----
target_pose = copy.deepcopy(T_fk[8])
print(target_pose)

possible_q_list = []
working_space = 0.6
n_point = 10
x_points = np.linspace(-working_space,working_space,n_point) + target_pose[0,3]
y_points = np.linspace(-working_space,working_space,n_point) + target_pose[1,3]
rot_points = np.linspace(-pi,pi,n_point)

T_agv_2_arm = now_agv_arm.T_agv_2_arm

for now_x in x_points:
    for now_y in y_points:
        for now_theta in rot_points:
            now_q = [0 for i in range(9)]
            now_q[0:2] = [now_x,now_y]
            now_q[2] = now_theta
            T_fk = now_agv_arm.FK(now_q)
            arm_T_06 = np.linalg.inv(T_fk[2] @ T_agv_2_arm) @ target_pose
            qs = jaka_ik_cal.IK(arm_T_06)
            for next_q in qs:
                now_q[3:] = next_q
                possible_q_list.append(copy.deepcopy(now_q))

print(len(possible_q_list))


[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  8.03030000e-01]
 [ 0.00000000e+00  1.10196153e-05 -1.00000000e+00  9.51958309e-03]
 [ 0.00000000e+00  1.00000000e+00  1.10196153e-05  2.03150888e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
5416


In [21]:
residual_list = []
rot_angle_list = []
for i in range(len(possible_q_list)):
    T_fk = now_agv_arm.FK(possible_q_list[i])
    residual = np.linalg.norm(T_fk[-1] @ np.linalg.inv(target_pose) - np.eye(4))
    residual_list.append(residual)

    rot_angle_list.append(np.sum(np.abs(possible_q_list[i][3:])))

residual_list = np.array(residual_list)
rot_angle_list = np.array(rot_angle_list)
th_error = 0.01
possible_index = np.where(residual_list < th_error)[0]

min_index = possible_index[np.argmin(rot_angle_list[possible_index])]
# min_index=  np.argmin(rot_angle_list)
print(f"min residual {residual_list[min_index]}")
T_fk = now_agv_arm.FK(possible_q_list[min_index])
print(T_fk[-1] @ np.linalg.inv(target_pose) - np.eye(4))
now_agv_arm.vis_collision()



    

min residual 0.0015079819903861924
[[-1.36300971e-11  6.43545837e-11  5.22105178e-06  1.09191398e-03]
 [-1.10515804e-10 -3.90859567e-11  8.84141187e-06  1.03986888e-03]
 [-5.22105178e-06 -8.84141188e-06 -5.27152766e-11  1.39781594e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
